In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.types import *
from delta import *

In [10]:
builder = SparkSession.builder.appName("olist_demo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.executor.memory", "2g")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.memory", "4g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Config

In [11]:
environment = '' # dev;uat;prd

In [12]:
path = {
    'brz': {'order_items': f'../../../{environment}/data/bronze/order_items'}, 
    'snk': '../../../data/silver/evt/evt_order_items'
}

## Load

In [13]:
df = spark.read.format('delta').load(path['brz']['order_items'])

In [14]:
df_snk = df.select(
    F.col("order_id").alias("id_order"),
    F.col("order_item_id").alias("nr_order_item"),
    F.col("product_id").alias("id_product"),
    F.col("seller_id").alias("id_seller"),
    F.col("shipping_limit_date").alias("dh_shipping_limit"),
    F.col("price").alias("vl_item"),
    F.col("freight_value").alias("vl_freight")
)

In [15]:
df_snk = df_snk.withColumn("dh_insert_slv", F.current_timestamp())

## Sink

In [16]:
df_snk.write.format('delta')\
.mode(saveMode='overwrite')\
.save(path['snk'])

24/10/03 15:47:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
